# Checagem de vantagens transitórias para Nereu

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from utils_ccd import get_connection, get_file_path, extract_text_from_pdf

conn = get_connection()

In [147]:
implementacoes = pd.read_excel('implementacoes_nereu.xlsx')
implementacoes['numero_processo'] = implementacoes['numero_processo'].apply(lambda x: str(x).zfill(6))
implementacoes['processo'] = implementacoes['numero_processo'] + '/' + implementacoes['ano_processo'].astype(str)

In [148]:
implementacoes['data_implementacao'] = pd.to_datetime(implementacoes['data_implementacao'], format='%d/%m/%Y', errors='coerce').dt.strftime('%m/%Y')


In [149]:
implementacoes.head()

,numero_processo,ano_processo,valor_implementado,data_implementacao,processo_sei,relator,obs,processo
0,003661,2022,1292.41,01/2025,0110012.003143/2024-18,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 3269/2023 para impl...,003661/2022
1,003269,2023,1235.29,01/2025,00110012.003144/2024-62,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 3661/2022 para impl...,003269/2023
2,001391,2023,1246.59,02/2025,0110012.003135/2024-71,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1418/2023 para impl...,001391/2023
3,001418,2023,1213.76,02/2025,110012.003138/2024-13,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1391/2023 para impl...,001418/2023
4,003666,2022,1281.37,03/2025,0110012.003140/2024-84,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 99/2023 para implem...,003666/2022


In [150]:
sql_nereu = f'''
SELECT DISTINCT 
        pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.codigo_tipo_processo,
       (
        SELECT CONCAT(numero_processo, '/', ano_processo)
        FROM processo.dbo.Processos
        WHERE IdProcesso = ed.IdProcessoOrigem
       ) as processo_origem,
       ed.IdProcessoExecucao as id_processo_execucao,
       (
       SELECT CONCAT(numero_processo, '/', ano_processo)
        FROM processo.dbo.Processos
        WHERE IdProcesso = ed.IdProcessoExecucao
       ) as processo_execucao
	FROM processo.dbo.Processos pro
	INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
	INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
	INNER JOIN processo.dbo.Relator r ON r.codigo = pro.codigo_relator 
	LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
	LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
	LEFT JOIN processo.dbo.Exe_TipoDebito etd ON etd.CodigoTipoDebito = ed.CodigoTipoDebito
	LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
	LEFT JOIN processo.dbo.Exe_Debito_MultaCominatoria edmc ON ed.IdDebito = edmc.IdDebito
	WHERE pro.setor_atual = 'CCD'
	AND CONCAT(pro.numero_processo, '/', pro.ano_processo) IN ({','.join(["'" + x + "'" for x in implementacoes['processo'].tolist()])})
'''
df_vantagens = pd.read_sql(sql_nereu, conn)

/tmp/ipykernel_87762/2322257977.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vantagens = pd.read_sql(sql_nereu, conn)


In [151]:
processos_origem = df_vantagens['processo_origem'].dropna().unique()

In [152]:
processos_origem

array(['011198/2017', '014157/2016', '015126/2016', '001390/2017',
       '012528/2016', '012961/2016', '023839/2016', '011818/2017',
       '006180/2016', '003493/2017', '006602/2016', '000337/2016',
       '000348/2016', '003205/2017', '009063/2017', '012540/2017',
       '012706/2016', '016807/2017', '005209/2016', '017667/2016',
       '003279/2017', '016419/2017', '026556/2016', '006161/2016',
       '014151/2016', '020069/2016', '024961/2016', '003275/2017',
       '004160/2016', '010477/2017', '008568/2017', '006600/2016',
       '023221/2016', '024593/2016', '003226/2017', '003282/2017',
       '003383/2017', '014683/2017', '003235/2017', '006949/2016',
       '014113/2017', '024690/2016'], dtype=object)

In [153]:
processos_busca = ', '.join(f"'{proc}'" for proc in processos_origem)

sql_all_informacoes_processos = f'''
SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
ppe.SequencialProcessoEvento as evento,
CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
inf.*
FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
    ON inf.idinformacao = ppe.idinformacao
WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos_busca})
'''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_file_path, axis=1)


/tmp/ipykernel_87762/1466441553.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [154]:
all_informacoes_processos['resumo'].fillna('', inplace=True)

In [155]:
all_informacoes_processos[all_informacoes_processos['resumo'].str.contains('DECIS')]['resumo']

218     MANDADO DE INTIMAÇÃO PARA COMUNICAÇÃO DA DECIS...
294     MANDADO DE INTIMAÇÃO PARA COMUNICAÇÃO DA DECIS...
325                                        DECISÃO PADRÃO
331                                        DECISÃO PADRÃO
354     MANDADO DE INTIMAÇÃO PARA CONHECIMENTO DE DECI...
                              ...                        
2796    MANDADO DE CITAÇÃO COM VISTAS AO CUMPRIMENTO D...
2799    MANDADO DE INTIMAÇÃO PARA CONHECIMENTO DE DECI...
2811    MANDADO DE CITAÇÃO COM VISTAS AO CUMPRIMENTO D...
2816    MANDADO DE INTIMAÇÃO PARA CONHECIMENTO DE DECI...
2820    MANDADO DE CITAÇÃO COM VISTAS AO CUMPRIMENTO D...
Name: resumo, Length: 287, dtype: object

In [156]:
all_informacoes_processos[all_informacoes_processos['resumo'].str.contains('decis')]['resumo'].values

array(['PEDIDO DE RECONSIDERAÇÃO. CONHECIMENTO. ATO COMPLEXO. VANTAGENS TRANSITÓRIAS. NATUREZA PROPTER LABOREM.  DESPROVIMENTO DO RECURSO.\r\nI - O pedido de reconsideração está de acordo com o artigo 376 do Regimento Interno do Tribunal de Contas, motivo pelo qual deve ser conhecido.\r\nII - Não infirmadas as razões da decisão, por meio de argumentos ou fatos aptos a impor a sua alteração, o pedido de reconsideração não merece provimento.\r\nIII - Opina-se pela manutenção integral do acórdão recorrido.\r\n'],
      dtype=object)

In [157]:
decisoes = f'''
SELECT *
FROM processo.dbo.vw_ia_votos_acordaos_decisoes
WHERE concat(numeroprocesso, '/', anoprocesso) in ({processos_busca})
'''
df_decisoes = pd.read_sql(decisoes, conn)

/tmp/ipykernel_87762/2911930637.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decisoes = pd.read_sql(decisoes, conn)


In [158]:
df_decisoes.columns

Index(['idvoto', 'codigo_tipo_processo', 'descricao', 'TipoVoto',
       'VotoEscolhido', 'idInformacao_voto', 'INFO_arquivo_Voto',
       'IdComposicaoPauta', 'CodigoCamara', 'numero_sessao', 'ano_sessao',
       'DataSessao', 'numeroResultado', 'anoResultado', 'resultadoTipo',
       'IdApreciacao', 'idTipoRecurso', 'IdTipoVotacao', 'decisao',
       'idVotoPauta', 'idVotoDecisao', 'NomeRelator', 'Setor', 'ementa',
       'assunto', 'NumeroProcesso', 'AnoProcesso', 'NomeRelatorProcesso',
       'IdProcesso', 'OrgaoOrigem', 'interessado', 'Divergente_de_idVoto',
       'isVotoDivergente', 'IdVotoConcordado', 'Relatorio',
       'FundamentacaoVoto', 'Conclusao', 'texto_acordao', 'SetorVoto',
       'DescricaoTipoVoto'],
      dtype='object')

In [159]:
df_decisoes['processo'] = df_decisoes.apply(
    lambda x: f"{x['NumeroProcesso']}/{x['AnoProcesso']}", axis=1
)

In [160]:
df_decisoes[['processo', 'texto_acordao']]

,processo,texto_acordao
0,001390/2017,DECIDEM os Conselheiros do Tribunal de Contas ...
1,003226/2017,DECIDEM os Conselheiros do Tribunal de Contas ...
2,003235/2017,DECIDEM os Conselheiros do Tribunal de Contas ...
3,003275/2017,DECIDEM os Conselheiros do Tribunal de Contas ...
4,003279/2017,DECIDEM os Conselheiros do Tribunal de Contas ...
...,...,...
78,026556/2016,"Vistos, relatados e discutidos estes autos, A..."
79,003275/2017,"Vistos, relatados e discutidos estes autos, ac..."
80,003235/2017,"Vistos, relatados e discutidos estes autos, AC..."
81,003279/2017,"Vistos, relatados e discutidos estes autos, AC..."


In [161]:
decisoes_grp = df_decisoes.groupby('processo').agg({
    'texto_acordao': lambda x: '\n'.join(x.dropna().astype(str)),
    'ementa': lambda x: '\n'.join(x.dropna().astype(str))
}).reset_index()

In [162]:
decisoes_grp

,processo,texto_acordao,ementa
0,000337/2016,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
1,000348/2016,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
2,001390/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
3,003205/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
4,003226/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
5,003235/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
6,003275/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
7,003279/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
8,003282/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...
9,003383/2017,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...


In [163]:
decisoes_grp['verbas_transitorias'] = decisoes_grp['ementa'].apply(lambda x: 'SIM' if 'PROPTER LA' in x else 'NÃO')

In [164]:
df_vantagens

,numero_processo,ano_processo,assunto,codigo_tipo_processo,processo_origem,id_processo_execucao,processo_execucao
0,000092,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,011198/2017,576947,000092/2023
1,000093,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,014157/2016,576948,000093/2023
2,000094,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,015126/2016,576949,000094/2023
3,000096,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,001390/2017,576951,000096/2023
4,000098,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,012528/2016,576953,000098/2023
5,000099,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,012961/2016,576954,000099/2023
6,000100,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,023839/2016,576955,000100/2023
7,000103,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,011818/2017,577183,000103/2023
8,000105,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,006180/2016,577275,000105/2023
9,000106,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,EXE,003493/2017,577278,000106/2023


In [165]:
decisoes_grp = decisoes_grp.merge(df_vantagens[['processo_execucao', 'processo_origem']], how='left', left_on='processo', right_on='processo_origem', suffixes=('', '_vantagens'))


In [166]:
tab_nereu = implementacoes.merge(
    decisoes_grp,
    left_on='processo',
    right_on='processo_execucao',
    how='left'
)
tab_nereu.drop(columns=['processo_x', 'processo_y', 
                        'processo_execucao'], inplace=True)

In [167]:
tab_nereu

,numero_processo,ano_processo,valor_implementado,data_implementacao,processo_sei,relator,obs,texto_acordao,ementa,verbas_transitorias,processo_origem
0,003661,2022,1292.41,01/2025,0110012.003143/2024-18,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 3269/2023 para impl...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,014113/2017
1,003269,2023,1235.29,01/2025,00110012.003144/2024-62,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 3661/2022 para impl...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,014683/2017
2,001391,2023,1246.59,02/2025,0110012.003135/2024-71,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1418/2023 para impl...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,016419/2017
3,001418,2023,1213.76,02/2025,110012.003138/2024-13,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1391/2023 para impl...,"Vistos, relatados e discutidos estes autos, A...",EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,NÃO,006161/2016
4,003666,2022,1281.37,03/2025,0110012.003140/2024-84,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 99/2023 para implem...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,024690/2016
5,000099,2023,1225.15,03/2025,110012.003139/2024-50,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 3666/2022 para impl...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,012961/2016
6,001417,2023,1213.76,04/2025,110012.003137/2024-61,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1420/2023 para impl...,"Vistos, relatados e discutidos estes autos, A...",EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,026556/2016
7,001420,2023,1249.47,04/2025,0110012.003141/2024-29,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 1417/2023 para impl...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,014151/2016
8,000100,2023,1225.15,05/2025,0110012.003142/2024-73,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 106/2023 para imple...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,023839/2016
9,000106,2023,1252.20,05/2025,110012.003261/2024-26,CARLOS THOMPSON COSTA FERNANDES,somada à multa do processo 100/2023 para imple...,DECIDEM os Conselheiros do Tribunal de Contas ...,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,SIM,003493/2017


In [168]:
tab_nereu.rename(columns={
    'texto_acordao': 'textos_acordaos_processo',
    'ementa': 'ementas_decisoes_processo'
}, inplace=True)

In [169]:
tab_nereu.to_excel('processos_descontos_nereu.xlsx', index=False)